# Cab-Lab

Welcome to Cab_Lab demo. In this notebook, a demonstration on how to access the data cube is presented.

In [ ]:
from cablab import Cube
from cablab.cube import CubeDataAccess
from datetime import datetime

Specify the cube location. This is the directory where "data" directory and "cube.config" file are located.

In [ ]:
cube = Cube.open("/home/jovyan/work/datacube/cablab-datacube-0.2.3/low-res")

In [ ]:
cube.data.variable_names

In [ ]:
cube.info

##### Get the cube's data.

get(variable=None, time=None, latitude=None, longitude=None)

**variable** an variable index or name or an iterable returning multiple of these (var1, var2, ...)

**time:** a single datetime.datetime object or a 2-element iterable (time_start, time_end)

**latitude:** a single latitude value or a 2-element iterable (latitude_start, latitude_end)

**longitude:** a single longitude value or a 2-element iterable (longitude_start, longitude_end)

**return:** a dictionary mapping variable names --> data arrays of dimension (time, latitude, longitude)

In [ ]:
data = cube.data.get(variable=['c_emissions', 'precipitation'])

In [ ]:
emission = data[0]
precip=data[1]

In [ ]:
emission.shape

In [ ]:
precip.shape

 ** Getting a chunk of 1 variable, all available time steps, and 40 x 40 spatial grid points **:

In [ ]:
cube_data = cube.data.get('precipitation',None,(0,10),(0,10))
#precip_chunk = cube_data[0]
cube_data[0].shape

** Getting time-series at a single point of all variables for the entire period: **

In [ ]:
time_series = cube.data.get(None,None,51.34,8.23)
[var.shape for var in time_series]

##### Get complete geo-location at a specific time

In [ ]:
precip_specific_time = precip.isel(time=0)

##### Get complete time-series for the specified geo-location.

In [ ]:
precip_specific_area = precip.isel(lat=508,lon=1296)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
precip_specific_area.load()

##### Generate a time series

In [ ]:
precip_specific_area[precip_specific_area[:]<0]=float('NaN')
plt.plot(precip_specific_area);

##### Multivariate plots

In [ ]:
emission_specific_area = emission.isel(time=slice(0,457),lon=1296,lat=508)
precip_specific_area = precip.isel(time=slice(0,457),lon=1296,lat=508)
plt.plot(range(0,457),emission_specific_area,"-",precip_specific_area/3,"-");

In [ ]:
from mpl_toolkits.basemap import Basemap
import numpy as np

In [ ]:
lon = np.linspace(-180,180,1440)
lat = np.linspace(90,-90,720)
lons, lats = np.meshgrid(lon,lat)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.05,0.05,0.9,0.9])
m = Basemap(projection='kav7',lon_0=0,resolution=None)
m.drawmapboundary(fill_color='0.3')
im1 = m.pcolormesh(lons,lats,precip_specific_time,shading='flat',cmap=plt.cm.jet,latlon=True)
m.drawparallels(np.arange(-90.,99.,30.))
m.drawmeridians(np.arange(-180.,180.,60.))
cb = m.colorbar(im1,"bottom", size="5%", pad="2%")
ax.set_title('Precipitation')